In [13]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [14]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Romny,50.7510,33.4747,62.56,59,90,6.13,UA,2021-08-05 03:18:57
1,1,Alice,27.7523,-98.0697,84.31,92,90,5.75,US,2021-08-05 03:20:40
2,2,Alofi,-19.0595,-169.9187,73.29,78,92,6.91,NU,2021-08-05 03:20:40
3,3,Troitsko-Pechorsk,62.7084,56.1964,60.04,88,100,4.18,RU,2021-08-05 03:20:40
4,4,Hilo,19.7297,-155.0900,80.33,73,75,6.91,US,2021-08-05 03:20:41


In [15]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [16]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [27]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
ax_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
    # Assign the figure variable.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=temps, dissipating=False, max_intensity=300, point_radius=4)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 70


In [30]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Romny,50.7510,33.4747,62.56,59,90,6.13,UA,2021-08-05 03:18:57
3,3,Troitsko-Pechorsk,62.7084,56.1964,60.04,88,100,4.18,RU,2021-08-05 03:20:40
18,18,Port Alfred,-33.5906,26.8910,63.18,88,74,8.88,ZA,2021-08-05 03:20:45
19,19,Sao Joao Da Barra,-21.6403,-41.0511,63.46,88,19,11.01,BR,2021-08-05 03:20:47
21,21,Busselton,-33.6500,115.3333,61.05,68,0,6.26,AU,2021-08-05 03:20:47
23,23,Mandalgovi,45.7625,106.2708,68.36,41,0,9.98,MN,2021-08-05 03:20:48
25,25,Inhambane,-23.8650,35.3833,61.93,78,0,4.45,MZ,2021-08-05 03:19:22
27,27,Ballina,-28.8667,153.5667,68.18,38,0,6.93,AU,2021-08-05 03:20:49
28,28,Petropavlovsk-Kamchatskiy,53.0452,158.6483,66.79,32,75,20.13,RU,2021-08-05 03:20:49
33,33,Kamenka,51.3223,42.7678,64.94,65,32,2.44,RU,2021-08-05 03:20:50


In [31]:
preferred_cities_df.count()


City_ID       132
City          132
Lat           132
Lng           132
Max Temp      132
Humidity      132
Cloudiness    132
Wind Speed    132
Country       132
Date          132
dtype: int64

In [32]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Romny,UA,62.56,50.7510,33.4747,
3,Troitsko-Pechorsk,RU,60.04,62.7084,56.1964,
18,Port Alfred,ZA,63.18,-33.5906,26.8910,
19,Sao Joao Da Barra,BR,63.46,-21.6403,-41.0511,
21,Busselton,AU,61.05,-33.6500,115.3333,
23,Mandalgovi,MN,68.36,45.7625,106.2708,
25,Inhambane,MZ,61.93,-23.8650,35.3833,
27,Ballina,AU,68.18,-28.8667,153.5667,
28,Petropavlovsk-Kamchatskiy,RU,66.79,53.0452,158.6483,
33,Kamenka,RU,64.94,51.3223,42.7678,


In [45]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

hotels

{'html_attributions': [],
 'next_page_token': 'Aap_uEBg0hB2H4EP5En-4yW-1ILEEZEczIaXjMlu5WkZo4VkghSloDrxA_6b1sT2mwS407BQ62tdCjmgcaAwSXjwtFTznyIOdKM8WcfkjzL-K7H6gRexD5gw7mZ73Fk1qQgcyG7cjV2_nxHUnoy5ZSN_APXXcDIxv5epVGEMggwtAjPTYHzaI9h7IH-l6KvpSFMJ6h9JHoCT11xph7qzq0dcsmBrqGQWVfYluBuaR8PBIcr_zRC-xJ-Br__xrxm-3U0Ts64J8WVkZHvs4hHRYYUCAyjan_XbiXJCvPAwROaTPFLtbZCFmlAEYfANE-cZo8SRBMK4AnLuAYCqdX1SsvrKMahxqTWCEzzPl9k4Z1WM-0zE3frDsC3NsAe_l5-F5QW_C-ukT10eEjjrtgD84aSRduJYU0af45VQ1GthzAXqn87-nbzz06LD08jF',
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 48.8581126, 'lng': 2.3529277},
    'viewport': {'northeast': {'lat': 48.8594393802915,
      'lng': 2.354352980291503},
     'southwest': {'lat': 48.8567414197085, 'lng': 2.351655019708499}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'icon_background_color': '#909CE1',
   'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/hotel_pinlet',
   'nam

In [64]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
# Grab the first hotel from the results and store the name.
try:
    hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
except (IndexError):
        print("Hotel not found... skipping.")

In [66]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Romny,UA,62.56,50.7510,33.4747,Floriana
3,Troitsko-Pechorsk,RU,60.04,62.7084,56.1964,Pechora Gostinitsa
18,Port Alfred,ZA,63.18,-33.5906,26.8910,The Halyards Hotel
19,Sao Joao Da Barra,BR,63.46,-21.6403,-41.0511,Pousada Porto De Canoas
21,Busselton,AU,61.05,-33.6500,115.3333,Observatory Guest House
...,...,...,...,...,...,...
571,Bonavista,CA,61.12,48.6499,-53.1147,
576,Mala,PE,64.89,-12.6581,-76.6308,
581,Iturama,BR,64.33,-19.7281,-50.1956,
585,Carlisle,US,67.78,40.2015,-77.1889,


In [67]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [68]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [72]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [73]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [74]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))